# 04 — Apprendre à convertir du texte en valeur numérique

Avant de faire du Machine Learning, il est nécessaire que toutes les données soient en format numérique.
Ce notebook nous permet donc d'aborder cet aspect essentiel pour avant toute tâche de d'entraiement de modèle.


In [199]:
import pandas as pd
from pathlib import Path

In [200]:
fp = Path('../data/bitcoin_wire_int.csv')
if not fp.exists():
    raise FileNotFoundError(
        f"Fichier introuvable : {fp}. Vérifie que bitcoin_wire_int.csv est bien dans le dossier ../data/"
    )
df = pd.read_csv(fp)

## - Volumétrie

In [201]:
df.shape

(1000, 3)

## - Types de colonnes

In [202]:
df.dtypes

bitcoin address    object
wire amount        object
credit card         int64
dtype: object

## - Aperçu visuel du dataset

In [203]:
df.head()

,bitcoin address,wire amount,credit card
0,1J9DVMtgvnPv4KGHL2XSzoTSt5cBBL1DXe,£8356.92,3543456448666398
1,1HaiVQbzCTs7t1K1fckYEUi9t8ERBFRR7Q,"€311,06",3584615208479876
2,1NGPjXfDScEUSZYajAzCkxaVApTbBHC7jP,"€2032,44",6370577480710876
3,15cSYRTbcrxYinxNxk7VbkPLZG8nypdjy6,¥8155.74,4508652385157398
4,18j2qG1DH9YduJiBiccAq8Ak897G3E4JyZ,£3265.44,374622922103592


In [204]:
df.sample(5, random_state = 42)

,bitcoin address,wire amount,credit card
521,114qsJYB8zRiTgn4jsjZZVuKRxNxFAnPAv,$5447.07,3553483082401582
737,1KKbGqnGzPHCnRAsXeftX7uKpTeGFrTPUe,"€5586,41",490556984931014332
740,15nBnpbkZK5mhd4hWdkdtRL7jtR2zTA61J,¥9381.24,3552476156649574
660,1PoQAoxN37yj26C7g92Uoar3bUx7pfwvG5,$5441.23,3578539326457267
411,171nNRsZVaEUK3ouwnrBzGCcx5dHhCdz7B,$7658.23,3541913323499578


In [205]:
df.tail()

,bitcoin address,wire amount,credit card
995,1Jmz4FMQENDkrtkivVdXM8SiC6ZCJVTDhp,¥844.68,5541649583285688
996,1GCZKgPV9277czn1RsTYoubCwhbndUvPZU,"€1433,63",6304232254235660027
997,1LK2DsWBUmSAsqg7n46sdgg4v2SNrqEPM,"€3814,11",3546750658508135
998,1PD6oNfFEnxMsR1PvUvkjHQdHYLCewLJhq,$2900.85,6333272716093574938
999,1B2vk6p7usuV4dfrBMLo8Rzpb5NatTmE8R,$1587.16,4913614335644981


## - Premier constat

Nous constatons plusieurs choses:
- que les valeurs dans la colonne `wire amount` sont de type `object` et pas du tout `numérique`.
- que certaines valeurs de cette colone sont préfixées par l'un des symboles monétaires suivants `£, €, $ ou ¥`.
- que, pour séparer la partie entière de la partie décimale, certaines valeurs contiennent le point `.` et d'autres la virgule `,`.

 Nous devons les convertir pour pouvoir les utiliser en machine learning.

In [206]:
df['currency'] = df['wire amount'].astype(str).str.strip().str[0]
df['wire amount'] = df['wire amount'].astype(str).str.strip().str[1:]
df['wire amount'] = df['wire amount'].str.replace(',', '.', regex=False)
df['wire amount'] = pd.to_numeric(df['wire amount'], errors = 'coerce')

## - Vérification de l'opération de conversion du type de la colonne 

In [207]:
df.dtypes

bitcoin address     object
wire amount        float64
credit card          int64
currency            object
dtype: object

Cette étape est indispensable avant toute modélisation :
les algorithmes de machine learning ne peuvent pas traiter des chaînes de caractères représentant des nombres.

In [208]:
df['wire amount'].isna().sum()
df.loc[df["wire amount"].isna()]

,bitcoin address,wire amount,credit card,currency


Un NaN généré par `errors="coerce"` est un signal d’alerte, pas une erreur à ignorer.

Les valeurs non convertibles doivent être analysées avant toute décision :
- amélioration du nettoyage,
- suppression des lignes concernées,
- ou imputation selon le contexte métier.

## - À retenir

- Une colonne peut contenir des nombres tout en étant typée `object`.
- Toujours vérifier les types avec `df.dtypes` après import.
- Nettoyer les symboles monétaires, espaces ou séparateurs avant conversion.
- Utiliser `pd.to_numeric(..., errors="coerce")` pour une conversion robuste.
- Vérifier les `NaN` générés après conversion.
- Toute donnée destinée au machine learning doit être numérique.